# Observability with Amazon Bedrock Converse API

The AWS Bedrock Converse API is a powerful tool for building conversational AI applications. It allows developers to create dynamic, multi-turn conversations with large language models (LLMs) hosted on AWS Bedrock. Here are some key benefits of using the Converse API:

1. **Stateful Conversations**: The API maintains conversation context across multiple turns, allowing for more natural and coherent interactions.

2. **Simplified Integration**: It provides a streamlined interface for integrating advanced language models into your applications without managing complex prompt engineering.

3. **Customization**: Developers can customize the conversation flow, system prompts, and model parameters to suit specific use cases.

4. **Scalability**: Being part of AWS Bedrock, it offers the scalability and reliability of AWS infrastructure.

5. **Access to Multiple Models**: You can easily switch between different LLMs available on AWS Bedrock without changing your application code.



In this notebook, we'll demonstrates how to implement observability for the Amazon Bedrock Converse API using the custom observability solution. We'll use the `BedrockLogs` class from the `observability` module to track and log API calls, responses, and user feedback.

### Prerequisite
After successfully setting up the backend resources required using the provided `CloudFormation template` to gather necessary data on user requests, your custom metadata like latency, time to first token, tags, model responses, citations, and any other custom identifiers you would like to add (e.g., user_id/customer_id), you can now test if your observability architecture is working as expected and determine the latency introduced by adding this additional component to your application.

#### `Important Note`: 

##### 1. Please use your AWS configuration to fill in the `config.py` file before running the code 

##### 2: Make sure you have upgraded your boto3 version to have at least `1.34.126` version.

### Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
import boto3
import json
import time
from datetime import datetime
import pytz
import string
import random
from uuid import uuid4

# Custom observability module
from observability import BedrockLogs

In [ ]:
# Import your configuration values (make sure to create a config.py file with these values)
from config import (
    REGION, FIREHOSE_NAME, CRAWLER_NAME, MODEL_ARN,
    EXPERIMENT_ID, CUSTOM_TAG, GUARDRAIL_ID, GUARDRAIL_VERSION,
    MAX_TOKENS, TEMPERATURE, TOP_P
)

In [ ]:
# Initialize BedrockLogs in Local mode with feedback variables
bedrock_logs = BedrockLogs(delivery_stream_name='local', feedback_variables=True)

In [ ]:
# Create AWS clients
boto3_session = boto3.session.Session()
bedrock_runtime_client = boto3.client('bedrock-runtime')

In [ ]:
# Helper function to generate a random session ID
def generate_web_session_id(length=16):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choices(characters, k=length))

### Main Function with Observability

Let's create our main function that uses the Converse API and is decorated with our observability logger. 

In [ ]:
@bedrock_logs.watch(call_type='Converse-API')
def converse_with_model(application_metadata):
    model_id = MODEL_ARN
    
    system_prompts = [{"text": "You are an expert that creates playlists for users. Only return song names and the artist."}]
    
    messages = [
        {
            "role": "user",
            "content": [{"text": application_metadata['question']}]
        }
    ]
    
    inference_config = {
        "temperature": TEMPERATURE,
        "maxTokens": MAX_TOKENS,
        "topP": TOP_P
    }
    
    guardrail_config = {
        "guardrailIdentifier": GUARDRAIL_ID,
        "guardrailVersion": GUARDRAIL_VERSION,
        "trace": "enabled"
    }
    
    response = bedrock_runtime_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        guardrailConfig=guardrail_config
    )
    
    application_metadata['model_response'] = response
    return response['output']['message']['content'][0]['text']

### Running the Conversation

Now, let's run a conversation with our model:

In [ ]:
question = "Create a list of 3 pop songs by artists from the United Kingdom."

In [ ]:
application_metadata = {
    'webSessionId': generate_web_session_id(),
    'userID': 'User-1',
    'customTags': CUSTOM_TAG,
    'request_time': datetime.now(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
    'model_arn': MODEL_ARN,
    'question': question
}

response, log, run_id, observation_id = converse_with_model(application_metadata)

print(f"Model response: {response}")
print(f"run_id: {run_id}")
print(f"observation_id: {observation_id}")

### Collecting Feedback

We'll define two functions for collecting feedback at the observation and session levels:

In [ ]:
@bedrock_logs.watch(call_type='observation-feedback')
def observation_level_feedback(feedback):
    pass

@bedrock_logs.watch(call_type='session-feedback')
def session_level_feedback(feedback):
    pass

user_feedback = 'Thumbs-up'
observation_feedback_from_front_end = {
    'user_id': 'User-1',
    'f_run_id': run_id,
    'f_observation_id': observation_id,
    'actual_feedback': user_feedback
}
observation_level_feedback(observation_feedback_from_front_end)

user_feedback = 'Amazing - this is fast and an awesome way to help the customers!'
session_feedback_from_front_end = {
    'user_id': 'User-1',
    'f_run_id': run_id,
    'actual_feedback': user_feedback
}
session_level_feedback(session_feedback_from_front_end)

# Conclusion

This notebook demonstrates how to use the customized `observability` module with the AWS Bedrock Converse API. We've shown how to:

1. Set up the environment and initialize the BedrockLogs class
2. Create a conversation function with observability
3. Run a conversation and collect the response
4. Implement feedback collection at both the observation and session levels

The collected data can be used for analysis, troubleshooting, and improving your application's performance and user experience.